In [11]:
import numpy as np
from collections import defaultdict
import random
import torch

class LSH:
    def __init__(self, L, h, dim):
        """
        Initialize the LSH tool.
        :param L: Number of layers
        :param h: Number of hashes per layer
        :param dim: Dimensionality of the input vectors
        """
        self.L = L
        self.h = h
        self.dim = dim
        self.hash_tables = [defaultdict(list) for _ in range(L)]
        self.random_vectors = [np.random.randn(h, dim) for _ in range(L)]
        self.offsets = [np.random.uniform(0.1, 1, h) for _ in range(L)]

    def _hash(self, vector, layer):
        """
        Compute the hash for a vector in a specific layer.
        :param vector: Input vector
        :param layer: Layer index
        :return: Hash value
        """
        projections = np.dot(self.random_vectors[layer], vector)
        return tuple(((projections + self.offsets[layer]) // 1).astype(int))

    def index(self, vectors):
        """
        Index a set of vectors.
        :param vectors: List of vectors to index
        """
        for idx, vector in enumerate(vectors):
            for layer in range(self.L):
                hash_value = self._hash(vector, layer)
                self.hash_tables[layer][hash_value].append(idx)
            print(f"Indexed vector {idx} with hash values: {[self._hash(vector, layer) for layer in range(self.L)]}")

    def query(self, vector, t):
        """
        Query the LSH index for the t most similar vectors.
        :param vector: Query vector
        :param t: Number of similar vectors to return
        :return: List of indices of the t most similar vectors
        """
        candidates = set()
        for layer in range(self.L):
            hash_value = self._hash(vector, layer)
            candidates.update(self.hash_tables[layer].get(hash_value, []))
        
        # Compute Euclidean distances to all candidates
        distances = [(idx, np.linalg.norm(vector - self.vectors[idx])) for idx in candidates]
        distances.sort(key=lambda x: x[1])
        
        return [idx for idx, _ in distances[:t]], len(candidates), len(distances)

In [3]:
FEATURE_SPACE = 'layer3'
FEATURE_FILE = '../data/extracted_features.pt'

In [4]:
# Load extracted features from Part 1
features = torch.load(FEATURE_FILE, weights_only=False)

selected_features = [elem[FEATURE_SPACE] for elem in features]
labels = [elem['class'] for elem in features]

In [16]:
# Index the features using LSH
lsh_f = LSH(L=2, h=2, dim=len(selected_features[0]))
lsh_f.index(selected_features)
# Store the features in the LSH instance for querying
lsh_f.vectors = np.array(selected_features)
# Query the LSH index with a random feature vector
# query_vector = np.random.rand(len(selected_features[0]))
random_index = random.randint(0, len(selected_features) - 1)
print(f"Randomly selected index for query: {random_index}")
query_vector = selected_features[random_index]
results, num_candidates, num_distances = lsh_f.query(query_vector, t=5)
# Output the results
print("Query Vector:", query_vector)
print("Indices of similar feature vectors:", results)
# Output the labels of the similar feature vectors
print("Labels of similar feature vectors:", [labels[idx] for idx in results])

/tmp/ipykernel_13552/4107731030.py:28: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  projections = np.dot(self.random_vectors[layer], vector)


Indexed vector 0 with hash values: [(np.int64(-1), np.int64(1)), (np.int64(-3), np.int64(0))]
Indexed vector 1 with hash values: [(np.int64(-1), np.int64(1)), (np.int64(-3), np.int64(1))]
Indexed vector 2 with hash values: [(np.int64(-2), np.int64(1)), (np.int64(-2), np.int64(0))]
Indexed vector 3 with hash values: [(np.int64(-1), np.int64(1)), (np.int64(-2), np.int64(0))]
Indexed vector 4 with hash values: [(np.int64(-2), np.int64(1)), (np.int64(-2), np.int64(-1))]
Indexed vector 5 with hash values: [(np.int64(-1), np.int64(1)), (np.int64(-3), np.int64(0))]
Indexed vector 6 with hash values: [(np.int64(-2), np.int64(1)), (np.int64(-2), np.int64(0))]
Indexed vector 7 with hash values: [(np.int64(-1), np.int64(1)), (np.int64(-3), np.int64(0))]
Indexed vector 8 with hash values: [(np.int64(-1), np.int64(1)), (np.int64(-3), np.int64(0))]
Indexed vector 9 with hash values: [(np.int64(-1), np.int64(1)), (np.int64(-2), np.int64(0))]
Indexed vector 10 with hash values: [(np.int64(-2), np.int6

/tmp/ipykernel_13552/4107731030.py:55: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  distances = [(idx, np.linalg.norm(vector - self.vectors[idx])) for idx in candidates]
